# Read Me

- This code cleans the original training data on Kaggle that has 55M rows. Because the dataset it big, so the code was written that can be run on Google Cloud. 
- The code sampled 25% of the cleaned data which was used in further analysis
- Cleaning specs include: 
1. NA 
2. Fare: 2.5<=Fare<=500
3. Drop-off/Pick-up location: has to be in NYC
4. Passenger count: 0<passenger count<=

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import math
import calendar
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime

#gcp dependencies
from google.cloud import storage
from io import BytesIO
client = storage.Client()
bucket = client.get_bucket("nytaxi_mz")

In [2]:
#read from gcp storage
blob = storage.blob.Blob("train.csv.zip",bucket)
content = blob.download_as_string()
train_data = pd.read_csv(BytesIO(content),compression='zip', header=0, sep=',', quotechar='"')
train_data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
#descriptives of train_data to spot any thing that requires cleaning
train_data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542386e+07,5.542386e+07,5.542386e+07,5.542348e+07,5.542348e+07,5.542386e+07
mean,1.134505e+01,-7.250968e+01,3.991979e+01,-7.251121e+01,3.992068e+01,1.685380e+00
std,2.071083e+01,1.284888e+01,9.642353e+00,1.278220e+01,9.633346e+00,1.327664e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


In [4]:
train_cleaned=train_data

#drop NAs
train_cleaned = train_cleaned.dropna()

#cleaned out the fare to make sure 2.5<fare<=500
#lowest fare of NYC taxi is 2.5
train_cleaned = train_cleaned[(train_cleaned['fare_amount']<=500) & (train_cleaned['fare_amount']>=2.5)]

In [5]:
#making sure all pick-up and drop-off are based inside New York City.
#New York City latitude and longitude range is: North Latitude: 40.917577 South Latitude: 40.477399 East Longitude: -73.700272 West Longitude: -74.259090
#Source: https://www.mapdevelopers.com/geocode_bounding_box.php
#Data out of the range will be cleaned out

train_cleaned = train_cleaned[(train_cleaned['pickup_longitude']<=-73.700272)& 
                              (train_cleaned['pickup_longitude']>=-74.259090)&
                              (train_cleaned['dropoff_longitude']<=-73.700272)& 
                              (train_cleaned['dropoff_longitude']>=-74.259090)&
                              (train_cleaned['pickup_latitude']<=40.917577)& 
                              (train_cleaned['pickup_latitude']>=40.477399)&
                              (train_cleaned['dropoff_latitude']<=40.917577)& 
                              (train_cleaned['dropoff_latitude']>=40.477399)]

In [6]:
#clean out passenger to make it 0<passenger<=9
train_cleaned = train_cleaned[(train_cleaned['passenger_count']<=9) & (train_cleaned['passenger_count']>0)]

#final clean data descriptives
train_cleaned.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.399245e+07,5.399245e+07,5.399245e+07,5.399245e+07,5.399245e+07,5.399245e+07
mean,1.129041e+01,-7.397550e+01,4.075085e+01,-7.397464e+01,4.075120e+01,1.691318e+00
std,9.462659e+00,3.445833e-02,2.689700e-02,3.385355e-02,3.073150e-02,1.307174e+00
min,2.500000e+00,-7.425903e+01,4.047759e+01,-7.425905e+01,4.047765e+01,1.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073657e+01,-7.399159e+01,4.073558e+01,1.000000e+00
50%,8.500000e+00,-7.398210e+01,4.075336e+01,-7.398062e+01,4.075384e+01,1.000000e+00
75%,1.250000e+01,-7.396835e+01,4.076752e+01,-7.396542e+01,4.076837e+01,2.000000e+00
max,5.000000e+02,-7.370028e+01,4.091754e+01,-7.370028e+01,4.091756e+01,9.000000e+00


In [ ]:
#sample 25% of the training data, and zip for easy sharing
sample_data = train_cleaned.sample(frac=.25)

#save the file as csv
sample_data.to_csv("train_data_cleaned_final.csv",compression='zip')

#save to gcp, needs to fix the code here
blob=bucket.blob('sample_data_cleaned.csv')
blob.upload_from_filename('sample_data_cleaned.csv')

In [ ]:
#export the final cleaned train data to csv then zip
train_cleaned.to_csv('train_data_cleaned.csv',compression='gzip')

#save to gcp, need to fix the code here
blob=bucket.blob('train_data_cleaned.csv')
blob.upload_from_filename('train_data_cleaned.csv')